# Big Data para Dados Públicos

##### Maciel C. Vidal

Na aula de hoje iremos conferir como criar, em um Jupyter Notebook, uma conexão com um SGBD para recuperar dados. Os dados desejados serão especificados utilizando SQL.


## Instalando as bibliotecas necessárias

Para criar uma conexão com o PostgreSQL, iremos utilizar a biblioteca `psycopg2`. Tente realizar o import dela e se não for possível, descomente e execute a seguinte linha para instalação:

In [ ]:
# !python -m pip install psycopg2-binary

Para criação de gráficos interativos, vamos utilizar a biblioteca `plotly.express`. Tente realizar o import dela e se não for possível, descomente e execute a seguinte linha para instalação:

In [ ]:
# !python -m pip install plotly_express

## Importando as bibliotecas necessárias

Agora, vamos importar as bibliotecas necessárias:

In [ ]:
# Para utilizar recursos do sistema
import os
import sys

# Para plot de gráficos
%matplotlib inline
import matplotlib.pyplot as plt
import plotly.express as px

# Para solicitar senha
import getpass

# Dataframe: tratar dados no Python
import pandas as pd

# Para conectar ao SGBD
import psycopg2 as pg


Caso obtenha algum erro, utilize o **!pip install** para instalar a biblioteca ausente!


Você também pode conferir de onde está executando o Python e qual a versão

In [ ]:
print("Executável:")
print(sys.executable)

print("\nVersão do Python:")
print(sys.version)

Vamos conferir em qual diretório iremos trabalhar (é o diretório do notebook)

In [ ]:
print("O seu notebook está na pasta:")
print(os.getcwd())

## Conectando à base de dados

O nosso objetivo final aqui é realizar alguma análise de dados (ex: extrair valores vendidos no mês, criar gráfico com quantidade de vendas). Como os dados estão armazenados em um SGBD, precisaremos criar uma conexão para permitir esta troca de informações:

<img src="https://bigdata-22-2.s3.us-east-2.amazonaws.com/sql/pandas_sql.png">

Para isso, primeiro iremos configurar algumas variáveis que irão conter as informações da conexão (caminho até o servidor, porta, database, usuário, senha):

In [ ]:
host = "18.117.69.137"
port = 5432
db = "aulas"
username = ""
# Para encontrar a senha: procure, no seu e-mail, por "feedbackinsper"
password = ""
# Se não quiser deixar a senha escrita no notebook,
# descomente a próxima linha. Será solicitado que informe a senha a cada execução.
# password = getpass.getpass("Digite a senha: ")

### Um primeiro exemplo de conexão

Para criar uma conexão, utilizaremos `pg.connect`, passando as informações necessárias (caminho até o servidor, porta, database, usuário, senha)

In [ ]:
conn = pg.connect(host=host, port=port, database=db, user=username, password=password)

cur = conn.cursor()

Agora a conexão existe e podemos utilizá-la para recuperar informações, por exemplo, sobre nossos clientes

In [ ]:
query = "SELECT * FROM olist.customer LIMIT 2"

cur.execute(query)

dados_clientes = cur.fetchall()

print(dados_clientes)

### Fechando a conexão

Quando a conexão não for mais necessária, é indicado que a conexão e o cursor sejam fechados, liberando recurso para os demais usuários.

In [ ]:
cur.close()
conn.close()

## Base de dados pública da Olist

Nesta aula, utilizaremos uma base pública, a **Brazilian E-Commerce Public Dataset by Olist**. Ela está disponível no **Kaggle** (https://www.kaggle.com/olistbr/brazilian-ecommerce), uma fonte legal para conseguir dados públicos e evoluir seus conhecimentos em Machine Learning.

<img src="https://bigdata-22-2.s3.us-east-2.amazonaws.com/sql/olist_db.png">

Os dados da base já foram baixados pelo professor e inseridos no SGBD PostgreSQL no **schema** **olist**. A relação entre as tabelas é:

<img src="https://bigdata-22-2.s3.us-east-2.amazonaws.com/sql/olist_der.png">

## Pandas

Para facilitar nosso trabalho com os dados tabulares extraídos do SGBD, iremos utilizar as bibliotecas Pandas e Dask para armazenar os dados em DataFrames. Uma outra facilidade é que o Pandas e Dask já possuem uma API para recuperar dados direto de SGBDs utilizando SQL

Vamos criar uma função que cria uma conexão, recupera dados e retorna um DataFrame:

In [ ]:
def get_pandas_df(sql):
    conn = pg.connect(host=host, port=port, database=db, user=username, password=password)
    
    df = pd.read_sql_query(sql, conn)
    
    conn.close()
    
    return df

Pronto! Agora podemos executar qualquer query e ter os resultados disponíveis!

In [ ]:
sql = """
SELECT id, city as cidade, state as uf
FROM olist.customer c
LIMIT 5
"""

df_cliente = get_pandas_df(sql)

Exibindo os resultados!

In [ ]:
dict(df_cliente.dtypes)

In [ ]:
df_cliente

## Pandas + SQLAlchemy

Apesar de funcionar, nas células anteriores você pode ter recebido um aviso que Pandas oficialmente suporta apenas conexões pela biblioteca **SQLAlchemy**. Vamos verificar como ficaria com esta biblioteca!

Caso necessário, descomente a próxima linha e faça a instalação

In [ ]:
# !python -m pip install sqlalchemy

Vamos importar a biblioteca

In [ ]:
import sqlalchemy

Verifique as alterações na função `get_pandas_df` para funcionar com sqlalchemy:

In [ ]:
def get_pandas_df(sql):
    engine = sqlalchemy.create_engine(f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{db}")

    df = pd.read_sql_query(sql, engine)

    engine.dispose()
    return df

E testar o mesmo exemplo utilizado anteriormente

In [ ]:
sql = """
SELECT *
  FROM olist.customer
 LIMIT 5
"""

df_cliente = get_pandas_df(sql)

df_cliente.head()

## Pandas: Lendo Tabela Completa

Com o uso da função `pd.read_sql_query` conseguimos executar qualquer query. Isto permite realizar *JOIN* entre tabelas, utilizar funções de agrupamento e agregação, etc. Entretanto, em certas situações já temos uma tabela ou *VIEW* com os dados disponíveis. Nestas situações, o Pandas disponibiliza a função `read_sql_table`.

Vamos ver um exemplo:

In [ ]:
engine = sqlalchemy.create_engine(f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{db}")

df_clientes = pd.read_sql_table("customer", engine, schema="olist")

engine.dispose()

df_clientes.head()

In [ ]:
def get_pandas_table(table, schema):
    engine = sqlalchemy.create_engine(f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{db}")
    df = pd.read_sql_table(table, engine, schema=schema)
    engine.dispose()
    return df

## Processando os dados: API DataFrame

Para processar dados provenientes de algum SGBD como o PostgreSQL, uma opção é ler os dados brutos de uma tabela e processar com a API de DataFrames do Pandas.

In [ ]:
df_cliente = get_pandas_table("customer", "olist")

Uma amostra dos dados:

In [ ]:
df_cliente.head(2)

Quantidade de linhas e colunas

In [ ]:
df_cliente.shape

Vamos realizar a contagem do número de clientes por cidade

In [ ]:
df_cliente["city"].value_counts(True).round(3).to_frame().head(6)

## Processando os dados com SQL

Uma outra opção é processar os dados utilizando uma query e recuperar apenas os resultados:

In [ ]:
# Preencha aqui
sql = """
SELECT c.city,
       ROUND(COUNT(c.city) /
               (SELECT count(1)::NUMERIC
                FROM olist.customer), 3) AS proportion
FROM olist.customer AS c
GROUP BY c.city
ORDER BY proportion DESC
LIMIT 6
"""

df_city_dist = get_pandas_df(sql)
df_city_dist

## Gerando gráficos com Plotly Express

Vamos utilizar o `plotly.express` para gerar gráficos interativos. Veja mais em https://plotly.com/python/plotly-express/

In [ ]:
sql = """
SELECT c.state AS UF,
       ROUND(COUNT(c.state) /
               (SELECT count(1)::NUMERIC
                FROM olist.customer), 3) AS freq_clientes
FROM olist.customer AS c
GROUP BY c.state
ORDER BY freq_clientes DESC
LIMIT 6
"""

df_uf_dist = get_pandas_df(sql)
df_uf_dist

In [ ]:
px.bar(df_uf_dist, x="uf", y="freq_clientes")

Vamos atualizar informações do gráfico como labels dos eixos para melhorar a exibição!

In [ ]:
# Alguns templates
# ["plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none"]

px.bar(df_uf_dist, x="uf", y="freq_clientes",
       labels={"freq_clientes": "Frequência", "uf": "Estado"},
       title="Distribuição dos clientes por UF", color="freq_clientes",
       template="ggplot2")

## Plot de Mapas com Folium

Vamos utilizar o `folium` para gerar gráficos interativos com informações georeferenciadas. Veja mais em https://python-visualization.github.io/folium/ e https://www.openstreetmap.org

Importe a biblioteca folium. Caso não seja possível, realize a instalação descomentando a seguinte célula:

In [ ]:
# !python -m pip install folium

Vamos importar as bibliotecas necessárias

In [ ]:
# Para mapas
import folium

# Para mapa de calor
from folium.plugins import HeatMap

Vamos exibir um primeiro mapa centrado na cidade de são paulo

In [ ]:
mapa = folium.Map(
    location=[-23.5489, -46.6388],
    zoom_start=12
)

mapa

Agora, vamos recuperar informações de alguns clientes (latitude e longitute) e plotar no mapa

In [ ]:
# recupera aleatoriamente 3% da base

sql = """
SELECT g.lat,
       g.lng
FROM olist.seller s,
     olist.geolocation g
WHERE s.zip_code_prefix = g.zip_code_prefix
  AND random() < 0.03
"""

Recuperando os dados

In [ ]:
df_heat = get_pandas_df(sql)
df_heat.head()

In [ ]:
df_heat.shape

Transformando em listas

In [ ]:
lat_list = df_heat["lat"].to_list()
lng_list = df_heat["lng"].to_list()
print(lat_list)

Exibindo o mapa de calor

In [ ]:
mapa = folium.Map(
    location=[-23.5489, -46.6388],
    zoom_start=4
)

HeatMap(list(zip(lat_list, lng_list))).add_to(mapa)

mapa

## Exercício 1

Identifique, na base olist.order_item, quais são os 20 produtos com mais vendas (valor total). Exiba um DataFrame contendo as informações do product_id e valor total faturado

In [ ]:
sql = """
 -- Sua query aqui
 """

df_top20 = get_pandas_df(sql)
df_top20

## Exercício 2

Exiba as informações do Exercício 1 em um gráfico de barras

## Exercício 3

Na base de ordens (`olist.order`), quais são os possíveis status para as compras?

## Exercício 4

Realize uma alteração na query do Exercício 1 para considerar apenas ordens com status entregue. Exiba um DataFrame e também um gráfico de barras

## Exercício 5

Considere apenas ordens com status entregue e sellers de SP. Encontre os top 5 produtos com maior faturamento (retorne o product_id e o faturamento total). Exiba um DataFrame e também um gráfico de barras

## Exercício 6

Considere apenas ordens com status entregue. Crie uma query SQL que calcule o faturamento mensal da olist em 2017. Exiba o DataFrame e um gráfico de linhas.

Dicas:
- Utilize `date_part` no SQL para extrair mês e ano https://www.postgresql.org/docs/13/functions-datetime.html
- px.line https://plotly.com/python/line-charts/

## Exercício 7

Considere apenas ordens com status entregue. Crie uma query SQL que calcule o faturamento mensal da olist em 2017 por UF do seller. Exiba o DataFrame (contento colunas de mês, UF e faturamento), além de um gráfico de linhas

Dicas:
- Utilize date_part no SQL para extrair mês e ano https://www.postgresql.org/docs/13/functions-datetime.html
- px.line https://plotly.com/python/line-charts/

## Exercício 8

Vamos exibir onde estão localizados os sellers da olist?!

Cruze as tabelas de seller e geolocation, e exiba um mapa de calor contendo aproximadamente 3% da base.